In [1]:
import sys
sys.path.append('/Applications/WhiteboxTools_darwin_m_series/WBT')  # Path to whitebox_tools.py
from whitebox_tools import WhiteboxTools
import rasterio
import numpy as np

In [2]:
wbt = WhiteboxTools()
wbt.set_working_dir('/Users/jpnousu/WBT_data')

In [3]:
# Reading raster data
dem_file = r'/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m.tif'
#dem = wbt.read_raster(dem_file)
#dem = wbt.fill_missing_data(dem, filter_size=35) # Notice that we overwrite `dem` here.
#streams_file = r'/Users/jpnousu/WBT_data/shapes/clipped_MTK-virtavesi_19-01-23_virtavesikapea.shp'
roads_file = r'/Users/jpnousu/WBT_data/shapes/clipped_MTK-tie_19-01-23_tieviiva.shp'
stream_o_file = r'/Users/jpnousu/WBT_data/shapes/stream_outlets.shp'
lakes_file = r'/Users/jpnousu/WBT_data/shapes/clipped_MTK-vakavesi_19-01-23_jarvi.shp'
#rivers_file = r'/Users/jpnousu/WBT_data/shapes/clipped_MTK-virtavesi_19-01-23_virtavesialue.shp'
streams_file = '/Users/jpnousu/WBT_data/shapes/merged_rivers_streams_2D.shp'

# reading shapes into vectors
#roads = wbt.read_vector(roads_file)
#streams = wbt.read_vector(streams_file)
#stream_o = wbt.read_vector(stream_o_file)
#lakes = wbt.read_vector(lakes_file)
#rivers = wbt.read_vector(rivers_file)

In [3]:
dem_file = r'/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m.tif'
roads_file = r'/Users/jpnousu/WBT_data/shapes/clipped_MTK-tie_19-01-23_tieviiva.shp'
streams_file = '/Users/jpnousu/WBT_data/shapes/merged_rivers_streams_2D.shp'
output = r'/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts.tif'

wbt.burn_streams_at_roads(
    dem_file, 
    streams_file, 
    roads_file, 
    output, 
    width=50.0, 
)

./whitebox_tools --run="BurnStreamsAtRoads" --wd="/Users/jpnousu/WBT_data" --dem='/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m.tif' --streams='/Users/jpnousu/WBT_data/shapes/merged_rivers_streams_2D.shp' --roads='/Users/jpnousu/WBT_data/shapes/clipped_MTK-tie_19-01-23_tieviiva.shp' --output='/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts.tif' --width='50.0' -v --compress_rasters=False

*********************************
* Welcome to BurnStreamsAtRoads *
* Powered by WhiteboxTools      *
* www.whiteboxgeo.com           *
*********************************
Reading streams and roads data...
Reading DEM raster...
Rasterizing Streams: 0%
Rasterizing Streams: 1%
Rasterizing Streams: 2%
Rasterizing Streams: 3%
Rasterizing Streams: 4%
Rasterizing Streams: 5%
Rasterizing Streams: 6%
Rasterizing Streams: 7%
Rasterizing Streams: 8%
Rasterizing Streams: 9%
Rasterizing Streams: 10%
Rasterizing Streams: 11%
Rasterizing Streams: 12%
Rasterizing Streams: 13%
Rasterizing Streams: 14%
Rasterizi

0

In [5]:
# Input files
water_file = r'/Users/jpnousu/WBT_data/pallas/hydro_4m.tif'
dem_file = r'/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts.tif'
output = r'/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts_water.tif'

# Drop amount in meters
drop = 1.0

# Open DEM
with rasterio.open(dem_file) as dem_src:
    dem_data = dem_src.read(1)
    dem_meta = dem_src.meta.copy()
    dem_nodata = dem_src.nodata

# Open hydro mask
with rasterio.open(water_file) as hydro_src:
    hydro_data = hydro_src.read(1)

# Create a mask for water pixels (non-zero in hydro mask)
water_mask = hydro_data > 0

# Make a copy to preserve original DEM
dem_modified = np.copy(dem_data)

# Drop elevation by 1m in water areas, avoiding nodata areas
if dem_nodata is not None:
    valid_mask = (dem_data != dem_nodata)
    dem_modified[water_mask & valid_mask] -= drop
else:
    dem_modified[water_mask] -= drop

# Save the modified DEM
dem_meta.update({
    'dtype': 'float32',
    'nodata': dem_nodata,
    'compress': 'lzw'
})

with rasterio.open(output, 'w', **dem_meta) as dst:
    dst.write(dem_modified.astype('float32'), 1)

print(f"Modified DEM saved to: {output}")

Modified DEM saved to: /Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts_water.tif


In [6]:
dem_file = r'/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts_water.tif'
output = r'/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts_water_no_debs.tif'

wbt.breach_depressions_least_cost(
    dem_file, 
    output, 
    dist=1000, 
    max_cost=1000.0,
)

./whitebox_tools --run="BreachDepressionsLeastCost" --wd="/Users/jpnousu/WBT_data" --dem='/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts_water.tif' --output='/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts_water_no_debs.tif' --dist='1000' --max_cost='1000.0' --min_dist --fill -v --compress_rasters=False

*****************************************
* Welcome to BreachDepressionsLeastCost *
* Powered by WhiteboxTools              *
* www.whiteboxgeo.com                   *
*****************************************
Reading data...
Finding pits: 0%
Finding pits: 1%
Finding pits: 2%
Finding pits: 3%
Finding pits: 4%
Finding pits: 5%
Finding pits: 6%
Finding pits: 7%
Finding pits: 8%
Finding pits: 9%
Finding pits: 10%
Finding pits: 11%
Finding pits: 12%
Finding pits: 13%
Finding pits: 14%
Finding pits: 15%
Finding pits: 16%
Finding pits: 17%
Finding pits: 18%
Finding pits: 19%
Finding pits: 20%
Finding pits: 21%
Finding pits: 22%
Finding pits: 23%
Finding pits: 24%
Finding pit

0

In [7]:
dem_file = r'/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts_water_no_debs.tif'
output = r'/Users/jpnousu/WBT_data/pallas/flowp_4m_culverts_water_no_debs.tif'

wbt.d8_pointer(
    dem_file, 
    output, 
)

./whitebox_tools --run="D8Pointer" --wd="/Users/jpnousu/WBT_data" --dem='/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts_water_no_debs.tif' --output='/Users/jpnousu/WBT_data/pallas/flowp_4m_culverts_water_no_debs.tif' -v --compress_rasters=False

****************************
* Welcome to D8Pointer     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progre

0

In [8]:
input_file = r'/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts_water_no_debs.tif'
output = r'/Users/jpnousu/WBT_data/pallas/flowacc_4m_culverts_water_no_debs.tif'

wbt.d8_flow_accumulation(
    input_file, 
    output, 
    out_type="specific contributing area", 
)

./whitebox_tools --run="D8FlowAccumulation" --wd="/Users/jpnousu/WBT_data" --input='/Users/jpnousu/WBT_data/pallas/korkeusmalli_4m_culverts_water_no_debs.tif' --output='/Users/jpnousu/WBT_data/pallas/flowacc_4m_culverts_water_no_debs.tif' --out_type=specific contributing area -v --compress_rasters=False

*********************************
* Welcome to D8FlowAccumulation *
* Powered by WhiteboxTools      *
* www.whiteboxgeo.com           *
*********************************
Reading data...
Flow directions: 0%
Flow directions: 1%
Flow directions: 2%
Flow directions: 3%
Flow directions: 4%
Flow directions: 5%
Flow directions: 6%
Flow directions: 7%
Flow directions: 8%
Flow directions: 9%
Flow directions: 10%
Flow directions: 11%
Flow directions: 12%
Flow directions: 13%
Flow directions: 14%
Flow directions: 15%
Flow directions: 16%
Flow directions: 17%
Flow directions: 18%
Flow directions: 19%
Flow directions: 20%
Flow directions: 21%
Flow directions: 22%
Flow directions: 23%
Flow direction

0

In [9]:
pour_pts = r'/Users/jpnousu/WBT_data/shapes/stream_outlets.shp'
output = r'/Users/jpnousu/WBT_data/shapes/stream_outlets_snap.shp'
flow_accum = r'/Users/jpnousu/WBT_data/pallas/flowacc_4m_culverts_water_no_debs.tif'

wbt.snap_pour_points(
    pour_pts, 
    flow_accum, 
    output, 
    snap_dist=20, 
)

./whitebox_tools --run="SnapPourPoints" --wd="/Users/jpnousu/WBT_data" --pour_pts='/Users/jpnousu/WBT_data/shapes/stream_outlets.shp' --flow_accum='/Users/jpnousu/WBT_data/pallas/flowacc_4m_culverts_water_no_debs.tif' --output='/Users/jpnousu/WBT_data/shapes/stream_outlets_snap.shp' --snap_dist='20' -v --compress_rasters=False

*****************************
* Welcome to SnapPourPoints *
* Powered by WhiteboxTools  *
* www.whiteboxgeo.com       *
*****************************
Reading data...
Progress: 0%
Progress: 25%
Progress: 50%
Progress: 75%
Progress: 100%
Saving data...
Output file written
Elapsed Time (excluding I/O): 0.0s


0

In [10]:
pour_pts = r'/Users/jpnousu/WBT_data/shapes/stream_outlets_snap.shp'
d8_pntr = r'/Users/jpnousu/WBT_data/pallas/flowp_4m_culverts_water_no_debs.tif'
output = r'/Users/jpnousu/WBT_data/pallas/ws_4m_culverts_water_no_debs.tif'
wbt.watershed(
    d8_pntr, 
    pour_pts, 
    output, 
)

./whitebox_tools --run="Watershed" --wd="/Users/jpnousu/WBT_data" --d8_pntr='/Users/jpnousu/WBT_data/pallas/flowp_4m_culverts_water_no_debs.tif' --pour_pts='/Users/jpnousu/WBT_data/shapes/stream_outlets_snap.shp' --output='/Users/jpnousu/WBT_data/pallas/ws_4m_culverts_water_no_debs.tif' -v --compress_rasters=False

****************************
* Welcome to Watershed     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Locating pour points: 0%
Locating pour points: 25%
Locating pour points: 50%
Locating pour points: 75%
Locating pour points: 100%
Initializing: 0%
Initializing: 1%
Initializing: 2%
Initializing: 3%
Initializing: 4%
Initializing: 5%
Initializing: 6%
Initializing: 7%
Initializing: 8%
Initializing: 9%
Initializing: 10%
Initializing: 11%
Initializing: 12%
Initializing: 13%
Initializing: 14%
Initializing: 15%
Initializing: 16%
Initializing: 17%
Initializing: 18%
Initializing: 19%
Initializing: 20%
Initializing: 21%
Initia

0

In [11]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape
import numpy as np

# File paths
input_raster = r'/Users/jpnousu/WBT_data/pallas/ws_4m_culverts_water_no_debs.tif'
output_shapefile = r'/Users/jpnousu/WBT_data/pallas/ws_4m_culverts_water_no_debs.shp'

# Open the raster
with rasterio.open(input_raster) as src:
    raster = src.read(1)
    mask = raster != src.nodata
    transform = src.transform
    crs = src.crs

# Extract shapes and values
results = (
    {'geometry': shape(geom), 'properties': {'ws_id': int(value)}}
    for geom, value in shapes(raster, mask=mask, transform=transform)
)

# Create GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(results, crs=crs)

# Save to shapefile
gdf.to_file(output_shapefile)

print(f"Watershed shapefile saved to: {output_shapefile}")

Watershed shapefile saved to: /Users/jpnousu/WBT_data/pallas/ws_4m_culverts_water_no_debs.shp
